In [ ]:
!pip install transformers datasets evaluate

In [ ]:
! pip install -q evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers[torch] datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset
import numpy as np
import evaluate
import os


In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
data_files = {
    "train": "ref_data.tsv",
    "validation": "dev_mms.tsv"
}
dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'date', 'headline', 'article', 'abstract'],
        num_rows: 3570
    })
    validation: Dataset({
        features: ['id', 'date', 'headline', 'article', 'abstract'],
        num_rows: 680
    })
})


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(examples["abstract"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")


model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")



In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-small-finetuned-mms",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    generation_num_beams=5,
    fp16=True,
    logging_dir="./logs",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-42-c3492850d419>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,4.092807,0.109100,0.029800,0.094800,0.094300,19.982400
2,No log,3.995559,0.107200,0.030900,0.095700,0.095200,19.876500
3,4.427200,3.952527,0.105600,0.032700,0.095300,0.094900,19.938200
4,4.427200,3.933044,0.107100,0.034600,0.097400,0.097000,19.933800
5,4.196700,3.928780,0.110500,0.036300,0.100200,0.099800,19.914700


TrainOutput(global_step=1120, training_loss=4.295892170497349, metrics={'train_runtime': 1328.1441, 'train_samples_per_second': 13.44, 'train_steps_per_second': 0.843, 'total_flos': 4831499299848192.0, 'train_loss': 4.295892170497349, 'epoch': 5.0})